In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross

from qiskit_metal.renderers.renderer_gds.gds_renderer import QGDSRenderer

In [3]:
design = designs.DesignPlanar()

In [4]:
design.renderers.keys()

dict_keys(['hfss', 'q3d', 'gds', 'gmsh', 'elmer', 'aedt_q3d', 'aedt_hfss', 'skeleton'])

In [5]:
a_skeleton = design.renderers.skeleton

In [6]:
gui = MetalGUI(design)
design.overwrite_enabled = True

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

05:28PM 26s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Could not create pixmap from c:\usershris\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.png

05:28PM 26s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Could not create pixmap from c:\usershris\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.png



In [7]:
## Custom options for all the transmons
options = dict(
    pad_width = '425 um',
    pad_gap = '80 um',
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict( 
        a = dict(loc_W=+1,loc_H=+1), 
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        d = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 TransmonPockets
q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.55mm', pos_y='+0.0mm', gds_cell_name='FakeJunction_02', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.9mm', orientation = '90', gds_cell_name='FakeJunction_02', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.55mm', pos_y='+0.0mm', gds_cell_name='FakeJunction_01',**options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.9mm', orientation = '90', gds_cell_name='my_other_junction', **options))

options = Dict(
    meander=Dict(
        lead_start='0.1mm',
        lead_end='0.1mm',
        asymmetry='0 um')
)

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='50um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight='0.13mm',
            end_straight='0.13mm'
        ),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 90
cpw1 = connect('cpw1', 'Q1', 'd', 'Q2', 'c', '5.7 mm', f'+{asym}um', fillet='25um')
cpw2 = connect('cpw2', 'Q3', 'c', 'Q2', 'a', '5.6 mm', f'-{asym}um', flip=True, fillet='100um')
cpw3 = connect('cpw3', 'Q3', 'a', 'Q4', 'b', '5.5 mm', f'+{asym}um', fillet='75um')
cpw4 = connect('cpw4', 'Q1', 'b', 'Q4', 'd', '5.8 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

In [8]:
a_skeleton.write_qgeometry_table_names_to_file('./simple_output.txt')

1

In [9]:
a_skeleton.write_qgeometry_table_names_to_file('./simple_output_cpw1.txt',highlight_qcomponents=['cpw1'])

1

In [10]:
design.qgeometry.tables['junction']

,component,name,geometry,layer,subtract,helper,chip,width,hfss_inductance,hfss_capacitance,...,q3d_inductance,q3d_capacitance,q3d_resistance,q3d_mesh_kw_jj,gds_cell_name,aedt_q3d_inductance,aedt_q3d_capacitance,aedt_hfss_inductance,aedt_hfss_capacitance,skeleton_a_column_name
0,1,rect_jj,"LINESTRING (2.55000 -0.04000, 2.55000 0.04000)",1,False,False,main,0.02,10nH,0,...,10nH,0,0,0.000007,FakeJunction_02,1.000000e-08,0,1.000000e-08,0,a_default_value
1,2,rect_jj,"LINESTRING (0.04000 -0.90000, -0.04000 -0.90000)",1,False,False,main,0.02,10nH,0,...,10nH,0,0,0.000007,FakeJunction_02,1.000000e-08,0,1.000000e-08,0,a_default_value
2,3,rect_jj,"LINESTRING (-2.55000 -0.04000, -2.55000 0.04000)",1,False,False,main,0.02,10nH,0,...,10nH,0,0,0.000007,FakeJunction_01,1.000000e-08,0,1.000000e-08,0,a_default_value
3,4,rect_jj,"LINESTRING (0.04000 0.90000, -0.04000 0.90000)",1,False,False,main,0.02,10nH,0,...,10nH,0,0,0.000007,my_other_junction,1.000000e-08,0,1.000000e-08,0,a_default_value


In [11]:
q1.options.skeleton_a_column_name = 'q1 skeleton'
q2.options.skeleton_a_column_name = 'q2 skeleton'
q3.options.skeleton_a_column_name = 'q3 skeleton'
q4.options.skeleton_a_column_name = 'q4 skeleton'

gui.rebuild()
gui.autoscale()

design.qgeometry.tables['junction']

,component,name,geometry,layer,subtract,helper,chip,width,hfss_inductance,hfss_capacitance,...,q3d_inductance,q3d_capacitance,q3d_resistance,q3d_mesh_kw_jj,gds_cell_name,aedt_q3d_inductance,aedt_q3d_capacitance,aedt_hfss_inductance,aedt_hfss_capacitance,skeleton_a_column_name
0,1,rect_jj,"LINESTRING (2.55000 -0.04000, 2.55000 0.04000)",1,False,False,main,0.02,10nH,0,...,10nH,0,0,0.000007,FakeJunction_02,1.000000e-08,0,1.000000e-08,0,q1 skeleton
1,2,rect_jj,"LINESTRING (0.04000 -0.90000, -0.04000 -0.90000)",1,False,False,main,0.02,10nH,0,...,10nH,0,0,0.000007,FakeJunction_02,1.000000e-08,0,1.000000e-08,0,q2 skeleton
2,3,rect_jj,"LINESTRING (-2.55000 -0.04000, -2.55000 0.04000)",1,False,False,main,0.02,10nH,0,...,10nH,0,0,0.000007,FakeJunction_01,1.000000e-08,0,1.000000e-08,0,q3 skeleton
3,4,rect_jj,"LINESTRING (0.04000 0.90000, -0.04000 0.90000)",1,False,False,main,0.02,10nH,0,...,10nH,0,0,0.000007,my_other_junction,1.000000e-08,0,1.000000e-08,0,q4 skeleton


In [12]:
q1.delete()
q2.delete()
q3.delete()
q4.delete()

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.55mm', pos_y='+0.0mm', gds_cell_name='FakeJunction_02', skeleton_a_column_name='q1 skeleton 2', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.9mm', orientation = '90', gds_cell_name='FakeJunction_02', skeleton_a_column_name='q2 skeleton 2', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.55mm', pos_y='+0.0mm', gds_cell_name='FakeJunction_01', skeleton_a_column_name='q3 skeleton 2', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.9mm', orientation = '90', gds_cell_name='my_other_junction', skeleton_a_column_name='q4 skeleton 2', **options))

design.qgeometry.tables['junction']

,component,name,geometry,layer,subtract,helper,chip,width,hfss_inductance,hfss_capacitance,...,q3d_inductance,q3d_capacitance,q3d_resistance,q3d_mesh_kw_jj,gds_cell_name,aedt_q3d_inductance,aedt_q3d_capacitance,aedt_hfss_inductance,aedt_hfss_capacitance,skeleton_a_column_name
0,9,rect_jj,"LINESTRING (2.55000 -0.01500, 2.55000 0.01500)",1,False,False,main,0.02,10nH,0,...,10nH,0,0,0.000007,FakeJunction_02,1.000000e-08,0,1.000000e-08,0,q1 skeleton 2
1,10,rect_jj,"LINESTRING (0.01500 -0.90000, -0.01500 -0.90000)",1,False,False,main,0.02,10nH,0,...,10nH,0,0,0.000007,FakeJunction_02,1.000000e-08,0,1.000000e-08,0,q2 skeleton 2
2,11,rect_jj,"LINESTRING (-2.55000 -0.01500, -2.55000 0.01500)",1,False,False,main,0.02,10nH,0,...,10nH,0,0,0.000007,FakeJunction_01,1.000000e-08,0,1.000000e-08,0,q3 skeleton 2
3,12,rect_jj,"LINESTRING (0.01500 0.90000, -0.01500 0.90000)",1,False,False,main,0.02,10nH,0,...,10nH,0,0,0.000007,my_other_junction,1.000000e-08,0,1.000000e-08,0,q4 skeleton 2
